In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score
from tensorflow.keras.optimizers import Adam

In [2]:
dataframe = pd.read_csv("../site_snow.csv")
temp = dataframe.drop(["Apr (WE)","May", "May (WE)", "Jun", "Jun (WE)",'Site_Name','County'], axis=1)
water = temp.dropna()
water['new_feature'] = (water['Jan'] + water['Feb'] + water['Mar']) / water['Elev']

C:\Users\aaron\AppData\Local\Temp\ipykernel_13360\2766607415.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  water['new_feature'] = (water['Jan'] + water['Feb'] + water['Mar']) / water['Elev']


In [3]:
water = water.iloc[:,:].apply(lambda x: pd.to_numeric(x, errors='coerce')).astype('float32')

In [4]:
order = ['Elev','Lat','Lon','Water Year','Jan','Jan (WE)','Feb','Feb (WE)','Mar','Mar (WE)','new_feature','installed','Apr']
water = water[order]

In [5]:
X = water.iloc[:,0:12]
y = water.iloc[:,12]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [7]:
X_val = X_train[0:637]
y_val = y_train[0:637]
X_train = X_train[637:]
y_train = y_train[637:]

In [10]:
model = Sequential([
    InputLayer(input_shape=[12,]),
    Dense(200, activation='relu'),
    Dense(200, activation='relu'),
    Dense(200, activation='relu'),
    Dense(200, activation='relu'),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error')

model.fit(X_train, y_train, epochs=500, batch_size=60,validation_data=(X_val, y_val))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/500
43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 49324.8320 - val_loss: 864.6469
Epoch 2/500
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 899.8333 - val_loss: 825.8908
Epoch 3/500
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 850.2854 - val_loss: 829.3529
Epoch 4/500
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 794.9860 - val_loss: 930.2664
Epoch 5/500
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 808.8818 - val_loss: 752.3842
Epoch 6/500
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 674.6984 - val_loss: 766.5055
Epoch 7/500
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 725.7964 - val_loss: 666.5717
Epoch 8/500
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 634.7454 - val_loss: 569.8796
Epoch 9/500
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 591.0393 - val_loss: 530.6989
Epoch 10/500
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 498.6026 - val_loss: 401.5687
Epoch 11/500
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 404.2031 - val_loss: 278.7234
Epoch 12/500
43/4

In [11]:
MSE = model.evaluate(X_test, y_test)
RMSE = np.sqrt(MSE)
y_pred = model.predict(X_val)
MAE = mean_absolute_error(y_val, y_pred)
R2 = r2_score(y_val, y_pred)

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 161.5468 
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [12]:
print("MSE of Neural Network: ",MSE)
print("RMSE of Neural Network: ", RMSE)
print("MAE of Neural Network:", MAE)
print("R^2 of Neural Network:", R2)

MSE of Neural Network:  151.1274871826172
RMSE of Neural Network:  12.293392012891202
MAE of Neural Network: 9.39842
R^2 of Neural Network: 0.8208887714553478
